# Making selections of feature layers

In [1]:
import pandas as pd
import pickle
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
import numpy as np

In [2]:
selection = pd.read_csv('imagenetSelection.csv',sep=':',header=None)

In [3]:
selection[0].shape

(257,)

In [4]:
with open('//code/logs/cifarVGGfc8trainfeats.pkl','rb') as f:
    feats = pickle.load(f)

In [3]:
vt = VarianceThreshold()
vt.fit(feats)

VarianceThreshold(threshold=0.0)

In [4]:
indx = np.argsort(vt.variances_)

In [5]:
var = vt.variances_
totalVar = np.sum(var)

In [20]:
np.sum(var[:400])/totalVar

0.08754990444072791

In [6]:
with open('//code/logs/400highestvarianceFeats.pkl','wb') as f:
    pickle.dump(indx[:400],f)

# Select features based on class closeness

In [80]:
with open('//code/logs/cifar10_1perc_selection.pkl','rb') as f:
    onepercent = pickle.load(f)

In [81]:
with open('//code/logs/cifar10labels.pkl','rb') as f:
    labels = pickle.load(f)

In [89]:
labels = labels[onepercent==1]

In [90]:
feats = feats[onepercent==1]

In [91]:
feats_class = np.argmax(feats,1)
clss, counts = np.unique(feats_class[labels==0],return_counts=True)
clss[np.argmax(counts)]

404

In [92]:
selection = []
c = []
for i in range(10):
    feats_class = np.argmax(feats,1)
    clss, counts = np.unique(feats_class[labels==i],return_counts=True)
    countsInd = np.argsort(counts)[::-1]
    maxc = countsInd[:5]
    sel = clss[maxc]
    selection.append(sel)
    c.append(counts[maxc])    
print(selection)
print(c)

[array([404, 812, 895, 856, 783]), array([675, 817, 491, 482, 653]), array([ 80, 335,  10, 653,  73]), array([335, 653, 152, 680, 158]), array([351, 339, 274, 335, 101]), array([152, 190, 175, 167, 158]), array([335, 126,  26,  32,  43]), array([339, 268, 101, 856, 167]), array([510, 814, 914, 812, 653]), array([675, 856, 510, 867, 491])]
[array([96, 52, 23, 23, 15]), array([217,  25,  22,  21,  20]), array([43, 42, 23, 21, 19]), array([47, 25, 21, 18, 17]), array([76, 48, 35, 25, 24]), array([82, 32, 23, 22, 22]), array([106,  32,  21,  20,  16]), array([188,  34,  19,  17,  16]), array([139,  39,  34,  28,  25]), array([284,  55,  25,  11,   7])]


In [97]:
i=4
print(selection[i])
print(c[i])

[351 339 274 335 101]
[76 48 35 25 24]


In [75]:
rel10 = [404,675,80,653,351,152,335,339,510,675]
with open('//code/logs/rel10.pkl','wb') as f:
    pickle.dump(rel10,f)

In [79]:
rel20 = [404,812,
         675,817,
         80,10,
         653,680,
         351,274,
         152,167,
         335,126,
         339,268,
         510,814,
         675,856]
with open('//code/logs/rel20.pkl','wb') as f:
    pickle.dump(rel20,f)

# Select features based on direct classification weights

In [2]:
import tensorflow as tf

In [4]:
saver = tf.train.Saver()
with tf.train.MonitoredTrainingSession(
    checkpoint_dir=FLAGS.train_dir,
    hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
           tf.train.NanTensorHook(loss),
           _LoggerHook()],
    config=tf.ConfigProto(
        log_device_placement=FLAGS.log_device_placement)) as mon_sess:

    ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)
    if ckpt and ckpt.model_checkpoint_path:
      # Restores from checkpoint
      saver.restore(mon_sess, ckpt.model_checkpoint_path)

ValueError: No variables to save